In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
DATASET_CSABSTRUCT_DIR = './gdrive/Shareddrives/DATASETS/CSAbstruct/'
OUTPUT_MODEL_DIR = './gdrive/Shareddrives/MODELS/'

In [ ]:
!pip install transformers -q
!pip install sentence_transformers -q
!pip install datasets -q

     |████████████████████████████████| 4.7 MB 15.7 MB/s 
     |████████████████████████████████| 6.6 MB 60.1 MB/s 
     |████████████████████████████████| 101 kB 12.5 MB/s 
     |████████████████████████████████| 596 kB 94.0 MB/s 
     |████████████████████████████████| 85 kB 2.0 MB/s 
     |████████████████████████████████| 1.3 MB 72.8 MB/s 
     |████████████████████████████████| 365 kB 15.6 MB/s 
     |████████████████████████████████| 115 kB 89.9 MB/s 
     |████████████████████████████████| 141 kB 85.0 MB/s 
     |████████████████████████████████| 212 kB 81.3 MB/s 
     |████████████████████████████████| 127 kB 77.3 MB/s 


In [ ]:
import pandas as pd

In [ ]:
df_train = pd.read_parquet(DATASET_CSABSTRUCT_DIR + 'train.parquet')
df_dev = pd.read_parquet(DATASET_CSABSTRUCT_DIR + 'dev.parquet')
df_test = pd.read_parquet(DATASET_CSABSTRUCT_DIR + 'test.parquet')

In [ ]:
df_train.columns

Index(['id', 'sentence', 'subject_label', 'label_id'], dtype='object')

In [ ]:
df_train['label_id'].unique()

array([0, 1, 2, 3, 4])

In [ ]:
df_train[['subject_label', 'label_id']].drop_duplicates()

,subject_label,label_id
0,background,0
1,objective,1
2,method,2
4,result,3
67,other,4


In [ ]:
# same text but with different labels
print(f'{len(df_train.index)}')
mask = df_train.groupby('sentence')['label_id'].transform('nunique') > 1
df_train = df_train[~mask].copy()
print(f'{len(df_train.index)}')

11333
11250


In [ ]:
# same text but with different labels
print(f'{len(df_dev.index)}')
mask = df_dev.groupby('sentence')['label_id'].transform('nunique') > 1
df_dev = df_dev[~mask].copy()
print(f'{len(df_dev.index)}')

2026
2024


In [ ]:
# same text but with different labels
print(f'{len(df_test.index)}')
mask = df_test.groupby('sentence')['label_id'].transform('nunique') > 1
df_test = df_test[~mask].copy()
print(f'{len(df_test.index)}')

1349
1349


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("nreimers/MiniLM-L6-H384-uncased", model_max_length=512)

In [ ]:
def preprocess_function(examples):
  tokenized_sentence = tokenizer(examples["sentence"], truncation=True)
  tokenized_sentence['label'] = examples['label_id']
  return tokenized_sentence

In [ ]:
from datasets import Dataset

dataset_train = Dataset.from_pandas(df_train)
dataset_val = Dataset.from_pandas(df_dev)
dataset_test = Dataset.from_pandas(df_test)

In [ ]:
dataset_train = dataset_train.map(preprocess_function)
dataset_val = dataset_val.map(preprocess_function)
dataset_test = dataset_test.map(preprocess_function)

  0%|          | 0/11250 [00:00<?, ?ex/s]

  0%|          | 0/2024 [00:00<?, ?ex/s]

  0%|          | 0/1349 [00:00<?, ?ex/s]

In [ ]:
label2id = {'background': 0, 'objective': 1, 'method': 2, 'result': 3, 'other': 4}
id2label = {0: 'background', 1: 'objective', 2: 'method', 3: 'result', 4: 'other'}

In [ ]:
from transformers import AutoConfig, AutoModelForSequenceClassification

# Download configuration from huggingface.co and cache.
config = AutoConfig.from_pretrained("nreimers/MiniLM-L6-H384-uncased", num_labels=5, id2label=id2label, label2id=label2id)
model = AutoModelForSequenceClassification.from_config(config)

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=20,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_total_limit=2,
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_train,
    eval_dataset=dataset_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: id, sentence, label_id, subject_label, __index_level_0__. If id, sentence, label_id, subject_label, __index_level_0__ are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 11250
  Num Epochs = 20
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 14080


Epoch,Training Loss,Validation Loss
1,1.403200,1.301524
2,1.258900,1.193744
3,1.063800,1.184055
4,1.018200,1.179989
5,0.923800,1.210446
6,0.862400,1.239319
7,0.814100,1.306083
8,0.750500,1.309796
9,0.708900,1.434938
10,0.655600,1.416162


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: id, sentence, label_id, subject_label, __index_level_0__. If id, sentence, label_id, subject_label, __index_level_0__ are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 2024
  Batch size = 16
Saving model checkpoint to ./results/checkpoint-704
Configuration saved in ./results/checkpoint-704/config.json
Model weights saved in ./results/checkpoint-704/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-704/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-704/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: id, sentence, label_id, subject_label, __index_level_0__. If id, sentence, label_id, su

TrainOutput(global_step=14080, training_loss=0.6955467251214114, metrics={'train_runtime': 625.5813, 'train_samples_per_second': 359.665, 'train_steps_per_second': 22.507, 'total_flos': 870238786803096.0, 'train_loss': 0.6955467251214114, 'epoch': 20.0})

In [ ]:
predictions = trainer.predict(dataset_test)

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: id, sentence, label_id, subject_label, __index_level_0__. If id, sentence, label_id, subject_label, __index_level_0__ are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1349
  Batch size = 16


In [ ]:
from datasets import load_metric
import numpy as np

def compute_metrics(eval_pred):
  metric_acc = load_metric("accuracy")
  metric_p = load_metric("precision")
  metric_r = load_metric("recall")
  metric_f1_macro = load_metric("f1")
  metric_f1_micro = load_metric("f1")
    
  preds = eval_pred.predictions
  labels = eval_pred.label_ids
  predictions = np.argmax(preds, axis=-1)

  accuracy = metric_acc.compute(predictions=predictions, references=labels)
  #precision = metric_p.compute(predictions=predictions, references=labels)
  #recall = metric_r.compute(predictions=predictions, references=labels)
  f1_micro = metric_f1_micro.compute(predictions=predictions, references=labels, average="micro")
  f1_macro = metric_f1_micro.compute(predictions=predictions, references=labels, average="macro")
  
  return {'accuracy': accuracy, 'f1_macro': f1_macro, 'f1_micro': f1_micro}

In [ ]:
compute_metrics(predictions)

{'accuracy': {'accuracy': 0.6805040770941438},
 'f1_macro': {'f1': 0.6780197676889272},
 'f1_micro': {'f1': 0.6805040770941438}}

In [ ]:
OUTPUT_MODEL_DIR = './gdrive/Shareddrives/MODELS/'

model.save_pretrained(OUTPUT_MODEL_DIR + 'csabstruct_minilm_classification')

Configuration saved in ./gdrive/Shareddrives/MODELS/csabstruct_minilm_classification/config.json
Model weights saved in ./gdrive/Shareddrives/MODELS/csabstruct_minilm_classification/pytorch_model.bin
